# Supplementary - Spatial filtering applied to calcium imaging data

Conducts analyses related to Supplementary Figure S12.

In [ ]:
import sys
sys.path.append("..")

from main import *
from zebrafish import *

from scipy.stats import zscore, spearmanr
from scipy.stats import percentileofscore
from brainsmash.mapgen.base import Base

# Setting paths

In [ ]:
atlas_path = '/home/anleg84/Documents/Atlas/Mapzebrain/'
atlas = Mapzebrain(atlas_path)

top_directory = '/media/anleg84/KINGSTON1/Datasets/Geometry/'
datasets = identify_folders(top_directory, ['920nm'])

In [ ]:
mask_tectum = atlas.get_region_mask(22)
mask_tectum[:, :, :284] = 0 # Focusing on right hemisphere (x=284 is the saggital midplane)
nodes_tectum = np.load('../Files/nodes_tectum_right_sampled.npy')

# Computing eigenmode-gradient correlations with spatial filtering

### Part 1: Optic tectum

Here, we sweep the $\sigma$ parameter for spatial filtering applied to calcium imaging data from the optic tectum. For each $\sigma$ value, FC matrices from 12 larvae are computed. First, individual fish data is loaded, detrended, slightly gaussian filtered temporally, and spatially filtered at the level of individual neuron coordinates. Then, the neurons are mapped to tectal nodes using nearest neighbors, the fluorescence signals are averaged per node, and a correlation (FC) matrix is computed for pairwise correlations of node signals. FC matrices are finally stacked and averaged, the group-averaged gradients are calculated, and compared with tectal eigenmodes computed beforehand in another notebook.

In [ ]:
sigma_values = np.linspace(0, 50, 26, endpoint=True)

In [ ]:
mode_similarity_matrices_tectum = []
correlation_matrices_tectum = []

for sigma in tqdm(sigma_values):

    # Loading data and computing tectal correlations
    FC_matrices = []
    for i, folder in enumerate(datasets):

        # Loading data and detrending calcium time series
        data = load_data(folder)
        centroids = data['centroids_atlas']
        in_tectum = mask_tectum[centroids[:, 2], centroids[:, 1], centroids[:, 0]] > 0
        centroids_tectum = centroids[in_tectum]
        dff_tectum = data['timeseries'][in_tectum]
        dff_tectum = compute_dff_using_minfilter(dff_tectum, window=120, sigma1=3, sigma2=60)
        dff_tectum = filter_timeseries(dff_tectum, 2)

        # Spatial filtering
        if sigma > 0:
            dff_tectum = spatial_smoothing(dff_tectum, centroids_tectum, sigma=sigma)

        # Mapping neurons to tectal nodes
        nn, _ = find_nearest_neighbors(centroids_tectum, nodes_tectum)
        node_timeseries = np.zeros((nodes_tectum.shape[0], dff_tectum.shape[1]))
        for i in range(nodes_tectum.shape[0]):
            neurons_in_node = (nn == i) # Neurons whose nearest neighbor corresponds to the tectum node i
            if np.any(neurons_in_node):
                if np.sum(neurons_in_node) == 1:
                    node_timeseries[i] = dff_tectum[neurons_in_node]
                else:
                    node_timeseries[i] = np.mean(dff_tectum[neurons_in_node], axis=0)       
        FC = np.corrcoef(node_timeseries)
        FC_matrices.append(FC)
        
    FC_matrices = np.stack(FC_matrices, axis=0)
    
    # Computing tectal gradients
    FC = np.nanmean(FC_matrices, axis=0)
    FC[np.diag_indices(FC.shape[0])] = 0
    N_modes = FC.shape[0]
    gradients, _ = diffusion_mapping(np.abs(FC), n_components=N_modes)
    
    # Correlation gradients with eigenmodes
    vertices = np.load('../Files/tectum_vertices_right.npy') * 40
    vertices = np.stack([vertices[:, 2], vertices[:, 1], vertices[:, 0]], axis=1)
    eigenmodes = np.load('../Files/tectum_eigenmodes_right.npy')[1:]
    d = compute_distances(vertices, nodes_tectum)
    eigenmodes_ = eigenmodes[:, np.argmin(d, axis=0)]
    mode_similarity, _ = compute_mode_similarity_matrix(eigenmodes_[:30], gradients.T[:30], return_mapping=True)

    mode_similarity_matrices_tectum.append(mode_similarity)
    correlation_matrices_tectum.append(FC)

In [ ]:
scores = []
for m in mode_similarity_matrices:
    scores.append(np.mean(np.abs(np.diag(m))[:10]))

In [ ]:
np.save('../Results/supp_smoothing_corrs_tectum.npy', correlation_matrices_tectum)
np.save('../Results/supp_smoothing_mode_similarity_tectum.npy', mode_similarity_matrices_tectum)

### Part 2: Whole-brain

Similar to the cells above, we sweep the $\sigma$ parameter for spatial filtering applied to calcium imaging data from the whole brain. For each $\sigma$ value, FC matrices from 22 larvae are computed. First, individual fish data is loaded, detrended, and slightly gaussian filtered temporally. Then, neurons are mapped to network nodes located throughout the brain, and the node-averaged activity is spatially filtered. This slight difference is due to the large memory requirements of calculating pairwise distances over ~50,000 neurons. In practice, filtering over neuron coordinates vs node coordinates (that is, before vs after coarse-graining activity) yields virtually identical results. Following the coarse-graining and filtering, FC matrices are finally computed, stacked and averaged, the group-averaged gradients are calculated, and compared with whole-brain eigenmodes computed beforehand in another notebook.

In [ ]:
top_directory = '/media/anleg84/KINGSTON/Datasets/Networks2024/'
datasets = get_datasets(top_directory, ['920nm', 'wholebrain'])
region_centroids_hires = np.load('../Files/PaperNetworks/centroids_hires.npy')

In [ ]:
N_regions = int(region_centroids_hires.shape[0] / 2)

In [ ]:
sigma_values = np.linspace(0, 200, 21, endpoint=True)

In [ ]:
mode_similarity_matrices_wholebrain = []
correlation_matrices_wholebrain = []

for sigma in tqdm(sigma_values):
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
    
        matrices = []
        for folder in datasets:
        
            if any(identify_files(folder, ['motor.npy'])):

                # Loading data and detrending calcium activity
                data = load_data(folder)
                region_labels = data['region_labels']
                region_labels[:, 10] = 0
                in_brain = np.sum(region_labels, axis=1) > 0
                centroids = data['centroids_atlas'][in_brain] # Excluding neurons outside the brain
                dff = np.load(folder + 'dff.npy')[in_brain][:, :600] # Selecting first 10 minutes without stimulation

                # Mapping neurons to network nodes
                NN, _ = find_nearest_neighbors(centroids, region_centroids_hires)
                N_neurons, N_dark, N_motor = [], [], []
                region_series = np.zeros((N_regions * 2, dff.shape[1]))
                for i in range(N_regions * 2):
                    c = centroids[NN == i]
                    if np.any(c):
                        region_series[i] = gaussian_filter1d(np.mean(dff[NN == i], axis=0), 2)
    
                if sigma > 0:
                    region_series = spatial_smoothing(region_series, region_centroids_hires, sigma=sigma)
                        
                region_series = 0.5 * (region_series[:N_regions] + region_series[N_regions:]) # Averaging across both hemispheres
                matrix = np.corrcoef(region_series)
                matrices.append(matrix)

    FC = np.nanmean(np.stack(matrices), axis=0)
    FC[np.isnan(FC)] = 0
    excluded_hi = np.where(np.sum(FC, axis=0) == 0)[0]
    FC = np.delete(np.delete(FC, excluded_hi, axis=0), excluded_hi, axis=1)
    FC[np.diag_indices(FC.shape[0])] = 0

    eigenmodes = np.load('../Files/zebrafish_wholebrain_eigenmodes_single.npy')
    vertices = np.load('../Files/zebrafish_wholebrain_vertices_single.npy')
    nodes = region_centroids_hires
    N_modes = FC.shape[0]
    gradients, _ = diffusion_mapping(np.abs(FC), n_components=N_modes)
    d = compute_distances(vertices, nodes[:FC.shape[0]])
    eigenmodes_ = eigenmodes[:, np.argmin(d, axis=0)]
    mode_similarity, mapping = compute_mode_similarity_matrix(eigenmodes_[1:31], gradients.T[:30], return_mapping=True)

    mode_similarity_matrices_wholebrain.append(mode_similarity)
    correlation_matrices_wholebrain.append(FC)

In [ ]:
plt.figure(figsize=(5, 5), dpi=150)

plt.imshow(np.abs(mode_similarity_matrices_wholebrain[0]), cmap='Reds', vmin=0.1, vmax=0.75)
plt.ylabel('Geometric modes')
plt.xlabel('Functional gradients')

In [ ]:
np.save('../Results/supp_smoothing_corrs_wholebrain.npy', correlation_matrices_wholebrain)
np.save('../Results/supp_smoothing_mode_similarity_wholebrain.npy', mode_similarity_matrices_wholebrain)

# Plotting results

Now we load the results from above and plot them.

In [ ]:
correlation_matrices_tectum = np.load('../Results/supp_smoothing_corrs_tectum.npy')
mode_similarity_matrices_tectum = np.load('../Results/supp_smoothing_mode_similarity_tectum.npy')
correlation_matrices_wholebrain = np.load('../Results/supp_smoothing_corrs_wholebrain.npy')
mode_similarity_matrices_wholebrain = np.load('../Results/supp_smoothing_mode_similarity_wholebrain.npy')
order = np.argsort(nodes_tectum[:, 1])

In [ ]:
n_modes = 50

scores_tectum = []
for m in mode_similarity_matrices_tectum:
    scores_tectum.append(np.mean(np.diag(np.abs(m))[:n_modes]))

scores_wholebrain = []
for m in mode_similarity_matrices_wholebrain:
    scores_wholebrain.append(np.mean(np.diag(np.abs(m))[:n_modes]))

sigma_values_tectum = np.linspace(0, 50, 26, endpoint=True)
sigma_values_wholebrain = np.linspace(0, 200, 21, endpoint=True)

In [ ]:
plt.plot(sigma_values_tectum, gaussian_filter1d(scores_tectum, 1))
plt.plot(sigma_values_wholebrain, gaussian_filter1d(scores_wholebrain, 1))

Measuring at what $\sigma$ value the peak geometric correlations are obtained.

In [ ]:
sigma_tectum = np.linspace(0, 50, 26, endpoint=True)
sigma_tectum[np.argmax(gaussian_filter1d(scores_tectum, 1))]

In [ ]:
sigma_wholebrain = np.linspace(0, 200, 21, endpoint=True)
sigma_wholebrain[np.argmax(gaussian_filter1d(scores_wholebrain, 1))]

# Measuring the effect of filtering on the cutoff point in tectum

Now we evaluate how spatial filtering influences the cutoff point in the geometric eigenmodes.

#### Measuring eigenmode cutoff points from elbow curves

In [ ]:
from scipy.optimize import curve_fit

mode_similarity_matrices = np.load('../Results/supp_smoothing_mode_similarity_tectum.npy')
wavelengths = np.load('../Results/wavelengths_tectum.npy')

cutoffs = []
for i in range(mode_similarity_matrices.shape[0]):
    
    diagonal = np.abs(np.diag(mode_similarity_matrices[i]))

    x = np.arange(len(diagonal))
    y = diagonal
    initial_guess = [20, -1, 0]
    params, _ = curve_fit(piecewise_linear, x, y, p0=initial_guess)
    cutoff = int(np.round(params[0]))

    cutoffs.append(cutoff)

#### Translating cutoff points into connectivity radii

In [ ]:
x = np.load('../Results/supp_tectum_cutoff_x2.npy')
y = np.load('../Results/supp_tectum_cutoff_y2.npy')

(a, b), CI_a, CI_b = linear_regression_with_confidence_interval(x, y, n_iter=100000)

In [ ]:
predicted_radii = (wavelengths[cutoffs] - b) / a
print(predicted_radii)

In [ ]:
radii_upper = (wavelengths[cutoffs] - CI_b[0]) / CI_a[0]
radii_lower = (wavelengths[cutoffs] - CI_b[1]) / CI_a[1]

# Rendering figure

In [ ]:
fig = PaperFigure(figsize=(7, 2), dpi=300)

fig.set_tick_length(1)
fig.set_font_size(6)
fig.add_background()

# Adding axes ----------------------------------------------------------------------------

w = 0.75
pad = (4 - 5 * w) / 2
for i in range(3):
    fig.add_axes(f'corr_tectum{i}', (i * (w + pad), 0), w, w)
    fig.add_axes(f'corr_wholebrain{i}', (i * (w + pad), w + pad), w, w)

fig.add_axes('curves', (3.25, 0), 1.5, 1.5)
fig.add_axes('radii', (5.5, 0), 1.5, 1.5)

fig.set_line_thickness(0.5)

# Filling content ------------------------------------------------------------------------

for i in range(3):
    ax = fig.axes[f'corr_tectum{i}']
    ax.imshow(correlation_matrices_tectum[int(2*i)][order, :][:, order], cmap='Reds', vmin=0, vmax=0.5)
    ax.set_xticks([])
    ax.set_yticks([])

for i in range(3):
    ax = fig.axes[f'corr_wholebrain{i}']
    ax.imshow(correlation_matrices_wholebrain[int(2*i)], cmap='Reds', vmin=0, vmax=1)
    ax.set_xticks([])
    ax.set_yticks([])

ax = fig.axes['curves']
ax.plot(sigma_values_tectum, gaussian_filter1d(scores_tectum, 1), color='red', linewidth=0.75)
ax.plot(sigma_values_wholebrain, gaussian_filter1d(scores_wholebrain, 1), color='black', linewidth=0.75)
ax.set_xlim([0, 200])
ax.spines[['top', 'right']].set_visible(False)

ax = fig.axes['radii']
ax.plot(sigma_values_tectum, predicted_radii, color='black', linewidth=0.75)
ax.axhspan(radii_upper[0], radii_lower[0], facecolor='red', edgecolor='None', alpha=0.2, zorder=-10)
ax.set_ylim([52 - 20, 52 + 20])
ax.spines[['top', 'right']].set_visible(False)
ax.set_xlim([0, np.max(sigma_values_tectum)])

fig.save('../Figures/supp_smoothing_incomplete.svg')

fig.show()

### Printing manually annotated values

In [ ]:
np.mean(predicted_radii)

In [ ]:
np.std(predicted_radii)

In [ ]:
sigma_values_wholebrain

In [ ]:
sigma_values_tectum